<a href="https://colab.research.google.com/github/rixprakash/DS2002F24/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/rixprakash/DS2002F24

Cloning into 'DS2002F24'...
remote: Enumerating objects: 359, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 359 (delta 123), reused 101 (delta 64), pack-reused 167 (from 1)
Receiving objects: 100% (359/359), 17.59 MiB | 16.23 MiB/s, done.
Resolving deltas: 100% (166/166), done.


In [9]:
import pandas as pd
import json
import sqlite3
import os

# Load the CSV file
file_path = '/content/DS2002F24/dataproject1/Crime_Data.csv'
df = pd.read_csv(file_path)
dfbefore = df.copy()

# Function to add the new file for Exact Date Report
def add_exact_date_report_column(df):
    try:
        # Extract the date part (first 10 characters) from 'DateReported'
        df['ExactDateReport'] = df['DateReported'].str[:10]
    except Exception as e:
        print(f"Error adding 'ExactDateReport' column: {e}")
    return df

# Function to split Officer Name
def split_reporting_officer(df):
    try:
        # Split the 'ReportingOfficer' column by the comma
        df[['OfficerLastName', 'OfficerFirstName']] = df['ReportingOfficer'].str.split(', ', expand=True)
    except Exception as e:
        print(f"Error splitting 'ReportingOfficer' into 'OfficerFirstName' and 'OfficerLastName': {e}")
    return df

# Function to convert CSV file to JSON
def convert_csv_to_json(df, file_path):
    try:
        json_data = df.to_json(orient='records')
        json_file_path = file_path.replace('.csv', '.json')
        with open(json_file_path, 'w') as json_file:
            json_file.write(json_data)
        print(f"CSV file converted to JSON format and saved as {json_file_path}")
    except Exception as e:
        print(f"Error converting CSV to JSON: {e}")

# Function to convert CSV file to SQL
def convert_csv_to_sql(df, file_path, db_name='crime_data.db', table_name='crime_records'):
    try:
        # Connect to SQLite database (or create it if it doesn't exist)
        conn = sqlite3.connect(db_name)

        # Write the dataframe to the SQL database
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        conn.close()
        print(f"CSV file converted to SQL and saved in {db_name} database, table: {table_name}")
    except Exception as e:
        print(f"Error converting CSV to SQL: {e}")

# Function to summarize data
def summarize_data(df):
    print(f"Number of records: {len(df)}")
    print(f"Number of columns: {len(df.columns)}")
    print(f"Columns: {list(df.columns)}\n")

# Main function to let the user choose
def main():
    df_modified = add_exact_date_report_column(df)
    df_modified = split_reporting_officer(df_modified)
    choice = input("Do you want to convert the CSV file to JSON or SQL Database? Enter 'JSON' or 'SQL': ")

    if choice == 'JSON':
        convert_csv_to_json(df, file_path)
    elif choice == 'SQL':
        convert_csv_to_sql(df, file_path)
    else:
        print("Invalid choice! Please enter 'JSON' or 'SQL'.")

    print("Pre-modified Data Summary:")
    summarize_data(dfbefore)
    print("Post-modified Summary:")
    summarize_data(df_modified)

if __name__ == "__main__":
    main()


Do you want to convert the CSV file to JSON or SQL Database? Enter 'JSON' or 'SQL': JSON
CSV file converted to JSON format and saved as /content/DS2002F24/dataproject1/Crime_Data.json
Pre-modified Data Summary:
Number of records: 25111
Number of columns: 9
Columns: ['RecordID', 'Offense', 'IncidentID', 'BlockNumber', 'StreetName', 'Agency', 'DateReported', 'HourReported', 'ReportingOfficer']

Post-modified Summary:
Number of records: 25111
Number of columns: 12
Columns: ['RecordID', 'Offense', 'IncidentID', 'BlockNumber', 'StreetName', 'Agency', 'DateReported', 'HourReported', 'ReportingOfficer', 'ExactDateReport', 'OfficerLastName', 'OfficerFirstName']



In [8]:
import pandas as pd
import json
import sqlite3
import os

file_path = '/content/DS2002F24/dataproject1/sportscarprice.json'
with open(file_path, 'r') as f:
    df = json.load(f)

# Function to load JSON file
def load_json_file(file_path):
    try:
        with open(file_path, 'r') as json_file:
            json_data = json.load(json_file)
        df = pd.DataFrame(json_data)
        print(f"JSON file loaded successfully from {file_path}")
        return df
    except Exception as e:
        print(f"Error loading JSON file: {e}")
        return None

# Function to convert JSON (DataFrame) to CSV
def convert_json_to_csv(df, file_path):
    try:
        csv_file_path = file_path.replace('.json', '.csv')
        df.to_csv(csv_file_path, index=False)
        print(f"JSON file converted to CSV and saved as {csv_file_path}")
    except Exception as e:
        print(f"Error converting JSON to CSV: {e}")

# Function to convert JSON (DataFrame) to SQL
def convert_json_to_sql(df, file_path, db_name='json_data.db', table_name='json_records'):
    try:
        # Connect to SQLite database (or create it if it doesn't exist)
        conn = sqlite3.connect(db_name)

        # Write the dataframe to the SQL database
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        conn.close()
        print(f"JSON file converted to SQL and saved in {db_name} database, table: {table_name}")
    except Exception as e:
        print(f"Error converting JSON to SQL: {e}")

# Main function to let the user choose
def main():
    file_path = input("Enter the JSON file path: ")

    # Check if file exists
    if not os.path.exists(file_path):
        print("File does not exist!")
        return

    # Load JSON file into a DataFrame
    df = load_json_file(file_path)

    if df is not None:
        # Ask user for the conversion choice
        choice = input("Do you want to convert the JSON file to CSV or SQL Database? Enter 'CSV' or 'SQL': ")

        if choice == 'CSV':
            convert_json_to_csv(df, file_path)
        elif choice == 'SQL':
            convert_json_to_sql(df, file_path)
        else:
            print("Invalid choice! Please enter 'CSV' or 'SQL'.")
    else:
        print("Unable to load the JSON file. Please check the file format.")

if __name__ == "__main__":
    main()


Enter the JSON file path: /content/DS2002F24/dataproject1/sportscarprice.json
JSON file loaded successfully from /content/DS2002F24/dataproject1/sportscarprice.json
Do you want to convert the JSON file to CSV or SQL Database? Enter 'CSV' or 'SQL': CSV
JSON file converted to CSV and saved as /content/DS2002F24/dataproject1/sportscarprice.csv
